#### Early Onset Colorectal Cancer

In [2]:
import os
import json
import time
from pathlib import Path
from typing import List, Dict, Any, Tuple
import requests
import pandas as pd
import numpy as np
import pickle
import tarfile, gzip, shutil
from numpy.linalg import norm
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
from tqdm import tqdm
import zipfile

/Users/reuben/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
GDC_API = "https://api.gdc.cancer.gov"
PROJECTS = ["TCGA-COAD", "TCGA-READ"] # colon + rectum adenocarcinoma
PAGE_SIZE = 200

SESSION = requests.Session()
SESSION.headers.update({"Content-Type": "application/json"})

##### Pull data & check outputs

In [4]:
## Handle pagination in GDC's API

def _paged_post(endpoint: str, payload: dict, page_size: int = PAGE_SIZE):
    """Yield all records from a paginated GDC API POST request."""
    url, offset = f"{GDC_API}/{endpoint}", 0
    while True:
        r = SESSION.post(url, params={"size": page_size, "from": offset},
                         data=json.dumps(payload), timeout=120)
        r.raise_for_status()
        hits = r.json().get("data", {}).get("hits", [])
        if not hits:
            break
        yield from hits
        offset += page_size
        if offset >= r.json().get("data", {}).get("pagination", {}).get("total", 0):
            break

In [5]:
## Query GDC for list of all open-access mutation files (MAF) for COAD+READ

def find_open_maf_files(projects=None):
    """
    Fetch open-access masked somatic mutation (MAF) files for TCGA projects (COAD/READ by default).
    Returns a DataFrame with file, case, and sample metadata.
    """
    projects = projects or PROJECTS

    # Build filters
    filters = {
        "op": "and",
        "content": [
            {"op": "in", "content": {"field": f, "value": v}}
            for f, v in [
                ("cases.project.project_id", projects),
                ("files.data_category", ["Simple Nucleotide Variation"]),
                ("files.data_type", ["Masked Somatic Mutation"]),
                ("files.experimental_strategy", ["WXS"]),
                ("files.access", ["open"])
            ]
        ],
    }

    fields = [
        "file_id", "file_name", "md5sum", "created_datetime", "updated_datetime",
        "analysis.workflow_type",
        "cases.case_id", "cases.submitter_id", "cases.project.project_id",
        "cases.samples.sample_id", "cases.samples.sample_type", "cases.samples.submitter_id"
    ]
    payload = {"filters": filters, "fields": ",".join(fields), "format": "JSON"}

    # Collect all hits
    rows = []
    for hit in _paged_post("files", payload):
        base = {
            "file_id": hit.get("file_id"),
            "file_name": hit.get("file_name"),
            "md5sum": hit.get("md5sum"),
            "created_datetime": hit.get("created_datetime"),
            "updated_datetime": hit.get("updated_datetime"),
            "workflow_type": hit.get("analysis", {}).get("workflow_type"),
        }
        for case in hit.get("cases", []):
            for s in case.get("samples", [{}]):  # ensure at least one row
                rows.append({
                    **base,
                    "case_id": case.get("case_id"),
                    "case_submitter_id": case.get("submitter_id"),
                    "project": case.get("project", {}).get("project_id"),
                    "sample_id": s.get("sample_id"),
                    "sample_submitter_id": s.get("submitter_id"),
                    "sample_type": s.get("sample_type"),
                })

    return pd.DataFrame(rows).drop_duplicates(["file_id", "sample_id", "case_id"]).reset_index(drop=True)

In [6]:
## Use '/data' bult endpoint to actually download MAFs by GDC file_id
## MAFs are very large, use manifest metadata first to select subset of MAFs to download

def download_files(file_ids, out_dir: str = "data/maf"):
    """
    Download one or more files from the GDC /data endpoint into out_dir.
    Handles batching, filenames, and safe writes. Returns list of file paths.
    """
    os.makedirs(out_dir, exist_ok=True)
    out_dir = Path(out_dir)
    paths = []

    for i in range(0, len(file_ids), 20):  # fetch in small batches
        chunk = file_ids[i:i+20]
        url = f"{GDC_API}/data/{','.join(chunk)}"
        r = SESSION.get(url, stream=True, timeout=600)
        r.raise_for_status()

        fname = r.headers.get("Content-Disposition", "").split("filename=")[-1].strip('"') \
                or (f"gdc_download_{i}.tar.gz" if len(chunk) > 1 else f"{chunk[0]}.maf.gz")

        fpath = out_dir / fname
        with open(fpath, "wb") as f:
            for b in r.iter_content(1 << 20):  # 1 MB chunks
                if b: f.write(b)

        paths.append(fpath)
        time.sleep(0.5)

    return paths

In [7]:
## Use the 'cases' endpoint to retrieve age at diagnosis (in days, convert to years)

def fetch_case_ages(projects=None):
    """
    Fetch mean age_at_diagnosis (years) per case for given TCGA projects.
    Returns: DataFrame [case_id, submitter_id, project, age_at_diagnosis_years].
    """
    projects = projects or PROJECTS

    filters = {
        "op": "and",
        "content": [
            {"op": "in", "content": {"field": "project.project_id", "value": projects}},
            {"op": "exists", "content": {"field": "diagnoses.age_at_diagnosis"}},
        ],
    }
    fields = [
        "case_id", "submitter_id", "project.project_id", "diagnoses.age_at_diagnosis"
    ]
    payload = {"filters": filters, "fields": ",".join(fields), "format": "JSON", "expand": "diagnoses"}

    rows = [
        {
            "case_id": h["case_id"],
            "submitter_id": h.get("submitter_id"),
            "project": h["project"]["project_id"],
            "age_at_diagnosis_years": (
                dx["age_at_diagnosis"] / 365.25 if isinstance(dx.get("age_at_diagnosis"), (int, float)) else None
            ),
        }
        for h in _paged_post("cases", payload)
        for dx in h.get("diagnoses", [])
    ]

    return (
        pd.DataFrame(rows)
        .groupby(["case_id", "submitter_id", "project"], as_index=False)
        .agg(age_at_diagnosis_years=("age_at_diagnosis_years", "mean"))
    )

In [8]:
## Categorize early/late stages

def label_early_late(age_years: float) -> str:
    """
    Categorize age into early / late / unknown onset groups.
    """
    if pd.isna(age_years):
        return "unknown"
    if age_years <= 40:
        return "early_onset"
    if age_years >= 70:
        return "late_onset"
    return "unknown"

def build_age_table():
    """
    Fetch patient ages and label each as early/late/unknown onset.
    Returns DataFrame with: case_id, submitter_id, project,
    age_at_diagnosis_years, group
    """
    cases = fetch_case_ages()
    cases["group"] = cases["age_at_diagnosis_years"].apply(label_early_late)
    return cases

In [9]:
## List mutation files that exist for TCGA colorectal cancer

maf_manifest = find_open_maf_files()
maf_manifest.head()

,file_id,file_name,md5sum,created_datetime,updated_datetime,workflow_type,case_id,case_submitter_id,project,sample_id,sample_submitter_id,sample_type
0,0a7104af-8c37-4492-9e72-f1e3f379e103,2c1ba550-574e-4bc3-a564-87e52ba76961.wxs.aliqu...,90bcd9b86636796cd4ee359b12d33e50,2022-08-01T12:47:55.680122-05:00,2024-07-31T04:41:28.097165-05:00,Aliquot Ensemble Somatic Variant Merging and M...,e394e9ec-7288-4ede-9ef8-41b631d100dd,TCGA-AA-3715,TCGA-COAD,90be4505-6489-43d9-b536-e90418dae2ed,TCGA-AA-3715-10A,Blood Derived Normal
1,0a7104af-8c37-4492-9e72-f1e3f379e103,2c1ba550-574e-4bc3-a564-87e52ba76961.wxs.aliqu...,90bcd9b86636796cd4ee359b12d33e50,2022-08-01T12:47:55.680122-05:00,2024-07-31T04:41:28.097165-05:00,Aliquot Ensemble Somatic Variant Merging and M...,e394e9ec-7288-4ede-9ef8-41b631d100dd,TCGA-AA-3715,TCGA-COAD,a172cf89-8820-4a84-97ef-6731e7ae71e2,TCGA-AA-3715-01A,Primary Tumor
2,48a5f19e-ae8b-4055-a5ff-961601aad512,f5ecb5b9-f00d-4d32-a3bd-42f8a168032c.wxs.aliqu...,11c466303107209c41e325cd90ba5de6,2022-08-01T12:52:19.222452-05:00,2024-07-31T04:44:00.711820-05:00,Aliquot Ensemble Somatic Variant Merging and M...,ac6a3df9-41b1-4e5a-a15e-f44481384c23,TCGA-AA-3860,TCGA-COAD,c1ea307f-cf39-4b09-8185-da56c37ce0d9,TCGA-AA-3860-01A,Primary Tumor
3,48a5f19e-ae8b-4055-a5ff-961601aad512,f5ecb5b9-f00d-4d32-a3bd-42f8a168032c.wxs.aliqu...,11c466303107209c41e325cd90ba5de6,2022-08-01T12:52:19.222452-05:00,2024-07-31T04:44:00.711820-05:00,Aliquot Ensemble Somatic Variant Merging and M...,ac6a3df9-41b1-4e5a-a15e-f44481384c23,TCGA-AA-3860,TCGA-COAD,d89599b3-35c7-4c31-8d3a-694da890c165,TCGA-AA-3860-10A,Blood Derived Normal
4,976e5a9e-dfe4-42e0-95fe-d7a33c8eecbc,d66378d1-4ec6-468f-8127-094989b2be3d.wxs.aliqu...,97124a969100de1709514298b965319b,2022-08-01T12:44:41.822225-05:00,2024-07-31T04:50:04.683869-05:00,Aliquot Ensemble Somatic Variant Merging and M...,f38d49b7-6cad-4460-abd0-3685a3b2d941,TCGA-AA-3814,TCGA-COAD,77abfd72-97d2-4abd-8633-99bedfffbe1d,TCGA-AA-3814-01A,Primary Tumor


In [10]:
## Pull & categorize ages at diagnosis for those cases

age_df = build_age_table()
#age_df["group"].value_counts(dropna=False)
age_df.head()

,case_id,submitter_id,project,age_at_diagnosis_years,group
0,0011a67b-1ba9-4a32-a6b8-7850759a38cf,TCGA-DC-6158,TCGA-READ,70.751540,late_onset
1,01240896-3f3f-4bf9-9799-55c87bfacf36,TCGA-F4-6854,TCGA-COAD,77.404517,late_onset
2,016c9c14-4c88-49f5-a11a-dd4bc282f11e,TCGA-DC-5337,TCGA-READ,68.999316,unknown
3,01ad5016-f691-4bca-82a0-910429d8d25b,TCGA-AA-3561,TCGA-COAD,72.334018,late_onset
4,01f493d4-229d-47a6-baa8-32a342c65d01,TCGA-AA-A00O,TCGA-COAD,83.000684,late_onset


In [11]:
## Select best MAF per case, merge with age info, and report QC

# Deduplicate
manifest_dedup = (
    maf_manifest
      .assign(
          is_primary = maf_manifest["sample_type"].eq("Primary Tumor"),
          wf_score = maf_manifest["workflow_type"].str.contains("MuTect2", case=False, na=False).astype(int),
          created_dt = pd.to_datetime(maf_manifest["created_datetime"], errors="coerce"),
          updated_dt = pd.to_datetime(maf_manifest["updated_datetime"], errors="coerce")
      )
      .sort_values(
          ["case_id", "is_primary", "wf_score", "updated_dt", "created_dt"],
          ascending=[True, False, False, False, False]
      )
      .drop_duplicates("case_id", keep="first")
      .drop(columns=["is_primary", "wf_score", "created_dt", "updated_dt"])
)

# Merge with clinical ages
manifest_with_age = (
    manifest_dedup
        .merge(age_df[["case_id","age_at_diagnosis_years","group"]],
               on="case_id", how="left")
        .query("group in ['early_onset','late_onset']")
        .reset_index(drop=True)
)

# Quality control summary
n_total = maf_manifest["case_id"].nunique()
n_dedup = manifest_dedup["case_id"].nunique()
n_merged = manifest_with_age["case_id"].nunique()
n_with_age = manifest_with_age["group"].notna().sum()

print(f"Total unique cases in raw manifest:     {n_total}")
print(f"Cases after deduplication (1 per case): {n_dedup}")
print(f"Cases retained after merging w/ age_df: {n_merged}")
print(f"Cases with known group (early/late):    {n_with_age}")
print("\nGroup breakdown:")
print(manifest_with_age["group"].value_counts(dropna=False))

Total unique cases in raw manifest:     583
Cases after deduplication (1 per case): 583
Cases retained after merging w/ age_df: 265
Cases with known group (early/late):    265

Group breakdown:
group
late_onset     250
early_onset     15
Name: count, dtype: int64


In [12]:
## Select balanced subset of cases and download their MAFs

n_per_group = 15

# Randomly sample up to n_per_group per onset group
subset_strat = (
    manifest_with_age
    .dropna(subset=["age_at_diagnosis_years"])
    .groupby("group", group_keys=False)
    .apply(lambda g: g.sample(n=min(n_per_group, len(g)), random_state=1))
    .reset_index(drop=True)
)

# Check
print("Sample counts:")
print(subset_strat["group"].value_counts())
print(subset_strat.groupby(["group", "project"]).size())

# Download selected MAFs
file_ids = subset_strat["file_id"].tolist()

# Create local data directory structure
os.makedirs("data/maf/downloads", exist_ok=True)

# Download selected MAFs to local directory
download_paths = download_files(file_ids, out_dir="data/maf/downloads")

/var/folders/vk/yxskvl4x63dbqzfj_xfn_wvw0000gn/T/ipykernel_56612/670174335.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  manifest_with_age


Sample counts:
group
early_onset    15
late_onset     15
Name: count, dtype: int64
group        project  
early_onset  TCGA-COAD    11
             TCGA-READ     4
late_onset   TCGA-COAD    10
             TCGA-READ     5
dtype: int64


##### Hypothesis testing
###### Hypothesis: early-onset colorectal tumours have more evidence of the colibactin mutational signature than late-onset tumours